In [1]:
import requests
import openpyxl
from bs4 import BeautifulSoup
from dateutil import rrule
from datetime import datetime

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
nltk.download('stopwords')

from tqdm import tqdm, tqdm_gui, trange
from time import sleep

headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/69.0'})

wb = openpyxl.Workbook()
wb.create_sheet(title = 'Первый лист', index = 0)

sheet = wb['Первый лист']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ospan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stemmer = RussianStemmer()
def snowball_stemmer(text):
    return [stemmer.stem(word) for word in text.split()]

In [3]:
stop = stopwords.words('russian')
text = 'прививка вакцинация белорусский коррупцией сша  протасевич мамин экономиека политика соглашения микрорайоне властей лукашенко токаев зеленский китай путин байден'
[w for w in snowball_stemmer(text) if w not in stop]

['прививк',
 'вакцинац',
 'белорусск',
 'коррупц',
 'сша',
 'протасевич',
 'мамин',
 'экономиек',
 'политик',
 'соглашен',
 'микрорайон',
 'власт',
 'лукашенк',
 'тока',
 'зеленск',
 'кита',
 'путин',
 'байд']

In [4]:
print(snowball_stemmer(text))

['прививк', 'вакцинац', 'белорусск', 'коррупц', 'сша', 'протасевич', 'мамин', 'экономиек', 'политик', 'соглашен', 'микрорайон', 'власт', 'лукашенк', 'тока', 'зеленск', 'кита', 'путин', 'байд']


In [5]:
mylist=snowball_stemmer(text)

In [6]:
page_start = 1 #начало отчёта
page_counter = 2 #конец отчёта
a = '20210528' #начало отчёта
b = '20210530' #конец отчёта
t = 0

In [7]:
sheet.append(['№ пп', 'Дата новости', 'Наименование новости', 'Описание, текст новости', 'Ссылка на новость'])

In [8]:
#forbes
for i in range(page_start, page_counter+1):
    url = f'https://forbes.kz/news?page={i}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('a', class_='news__mini-info')
    for n, j in enumerate(items, start=1):
        try:
            itemName = j.text.strip()
            Name = itemName
            count = 0
            count = sum(s.lower() in word for word in Name.lower().split() for s in mylist)
            itemTag = j['href']
            itemTag = 'https://forbes.kz'+itemTag
            responseTag = requests.get(itemTag)
            soupTag = BeautifulSoup(responseTag.text, 'lxml')
            News = soupTag.find('article', class_='inner-news')
            News = News.find_all('p')
            TextNews = ''
            TextNews = TextNews.join(s.text for s in News)
            txt = " ".join(s for s in mylist if s.lower() in TextNews.lower())
            cheker = len(txt.split())
            if count>=3  or cheker >= 5:
                itemTime = soupTag.find('div', class_='article__date is-tcell').text.strip()
            else: 
                itemTime = ''
                itemName = ''
                itemTag = ''
                TextNews = ''
                n = n -1
        except AttributeError:         
            n = n -1
        else:
            if itemName != '':
                t = t + 1
                print(f'{t}: {itemTime} {itemName} {itemTag} ')
                print (TextNews)
                sheet.append([f'{t}', f'{itemTime}', f'{itemName}',f'{TextNews}',f'=HYPERLINK("{itemTag}", "{itemTag}")'])

#tengri
for i in range(page_start, page_counter+1):
    url = f'https://tengrinews.kz/news/page/{i}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', class_='tn-article-item')
    for n, j in enumerate(items, start=1):
        try:
            itemName = j.find('span', class_='tn-article-title').text.strip()
            Name = j.find('span', class_='tn-article-title').text.strip().lower()
            count = 0
            count = sum(s.lower() in word for word in Name.lower().split() for s in mylist)
            itemTag = j.find('a', href=True)['href'].strip()
            itemTag = 'https://tengrinews.kz'+itemTag
            responseTag = requests.get(itemTag)
            soupTag = BeautifulSoup(responseTag.text, 'lxml')
            TextNews = soupTag.find('article', class_='tn-news-text')
            TextNews = TextNews.find_all('p')
            News = TextNews[0:len(TextNews)-2]
            TextNews = ''
            TextNews = TextNews.join(s.text for s in News)           
            txt = " ".join(s for s in mylist if s.lower() in TextNews.lower())
            cheker = len(txt.split())
            if count>=3  or cheker >= 5:
                itemName = j.find('span', class_='tn-article-title').text.strip()
                itemTime = j.find('time').text.strip()
            else: 
                itemTime = ''
                itemName = ''
                itemTag = ''
                TextNews = ''
                n = n -1
        except AttributeError:         
            n = n -1
        else:
            if itemName != '':
                t = t + 1
                print(f'{t}: {itemTime} {itemName} {itemTag}')
                sheet.append([f'{t}', f'{itemTime}', f'{itemName}',f'{TextNews}',f'=HYPERLINK("{itemTag}", "{itemTag}")'])

# Zakon
for dt in rrule.rrule(rrule.DAILY,
                      dtstart=datetime.strptime(a, '%Y%m%d'),
                      until=datetime.strptime(b, '%Y%m%d')):
    time = dt.strftime('%Y%m%d')
    #print (time)
    time = f'{time[:4]}/{time[4:6]}/{time[6:8]}/'  
    url = f'https://www.zakon.kz/{time}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('a', class_='tahoma font12')   
    for n, j in enumerate(items, start=1):
        try:
            Name = j.text.lower() 
            count = 0
            count = sum(s.lower() in word for word in Name.lower().split() for s in mylist)
            itemTag = 'https://www.zakon.kz' + j['href']
            responseTag = requests.get(itemTag)
            soupTag = BeautifulSoup(responseTag.text, 'lxml')
            News = soupTag.find('div', class_='fullcont')
            News = News.find_all(['p', 'li', 'blockquote'])
            TextNews = ''
            TextNews = TextNews.join(s.text for s in News)
            txt = " ".join(s for s in mylist if s.lower() in TextNews.lower())
            cheker = len(txt.split())
            if count>=3  or cheker >= 5:
                itemName = j.text
            else: 
                itemName = ''
                itemTag = ''
                TextNews = ''
                n = n -1
        except AttributeError:
            n = n -1
        else:
            if itemName != '':
                t=t+1
                print(f'{t}: {time} {itemName} {TextNews} {itemTag}')
                sheet.append([f'{t}', f'{time}', f'{itemName}',f'{TextNews}',f'=HYPERLINK("{itemTag}", "{itemTag}")'])
                
        wb.save('example.xlsx')
        
for n in tqdm(range(100), 'Загрузка информации в эксель'):
    sleep(.05)        

1: 30 мая 2021 В Кремле раскрыли детали переговоров Путина и Лукашенко https://forbes.kz/news/2021/05/30/newsid_250904 
Пресс-секретарь президента России Дмитрий Песков рассказал о переговорах Владимира Путина с белорусским коллегой Александром Лукашенко в Сочи, передает РИА Новости. Глава республики прибыл в Краснодарский край накануне. В пятницу лидеры общались более пяти часов. В субботу переговоры продолжились.

Одной из тем встречи стало задержание россиянки Софьи Сапеги, которая 23 мая летела вместе с основателем Telegram-канала Nexta Романом Протасевичем на самолете Ryanair. Девушку арестовали на два месяца.

«Разумеется, ее судьба нам небезразлична. Но также принимая во внимание тот факт, что у нее действительно <...> есть вид на жительство в Беларуси», — заявил Песков журналистам.

Путин поручил МИД «самым внимательным образом отслеживать дело Сапеги, подчеркнув при этом, что «все должно происходить в рамках закона».Как отметил представитель Кремля, в ситуации с рейсом Ryanair

6: 2021/05/28/ Мамин предложил разработать Стратегию обеспечения продовольственной безопасности стран СНГ Премьер-министр РК Аскар Мамин в рамках визита в Минск принял участие в заседании Совета глав правительств (СГП) Содружества Независимых Государств (СНГ), на котором отметил необходимость формирования новой повестки устойчивого развития экономики СНГ в постпандемический период, сообщает zakon.kz со ссылкой на primeminister.kz.Страны СНГ последовательно переходят к фазе постпандемического восстановления. Наши экономики, по прогнозам ведущих международных финансовых организаций, в текущем году выйдут на траекторию положительного роста, — сказал Мамин.Глава правительства отметил улучшение показателей взаимной торговли между странами СНГ. В январе-мартеторговля Казахстана со странами СНГ составила $6,3 млрд, что на 10% больше, чем за аналогичный период прошлого года.Премьер-министр РК подчеркнул необходимость принятия скоординированных мер по обеспечению совместного формирования добавл

Загрузка информации в эксель: 100%|██████████████████████████████████████████████████| 100/100 [00:06<00:00, 16.04it/s]
